2) Se tiene un registro de transaccoines bancarias, de la forma (nro de trans, tipo de trans, cuenta origen, cuenta destino, fecha, hora, monto). Se pide resolver en pandas:
-A) validar que todas las transacciones cuenten con un tipo de transaccion
-B) validar que para las transacciones del tipo transferencia, exista siempre tanto cuenta origen como cuenta                          destino
-c) Verificar que odas las transacciones del tipo transefrencia, deposito y extraccion cuenten con montos distintos de cero
-D) Indicar cuales fueron las 10 transacciones de mayor monto
-E) Indicar cual es el tipo de transaccion con mayor monto promedio
-F) Indicar cuales son las 5 cuentas con mayor cantidad de transacciones
-G) Indicar cuales son las 5 cuentas con mayor monto involucrado
-H) Para el tipo de transaccion con mayor cantidad de monto promedio, indicar cuales son las 5 cuentas con mas transaacciones

In [1]:
import pandas as pd
import numpy as np
import random

#Creo un dataframe de prueba
datos = pd.DataFrame(columns = ['Num. de transaccion', 'Tipo de transaccion', 'Cuenta origen','Cuenta destino','Fecha','Hora','Monto'])
tipoTransaccion = ['Transferencia','Extraccion','Deposito',np.nan]
cuentas = ['Cuenta 1', 'Cuenta 2', 'Cuenta 3', 'Cuenta 4','Cuenta 5', np.nan]
fechas = ['7/6/2019','20/6/2019','23/9/2018','2/10/2019','5/1/2020']
horas = ['10:00:00','13:05:00','15:45:00','16:23:05']
for i in range(10):
    datos = datos.append({
        'Num. de transaccion' : (i+15),
        'Tipo de transaccion' : random.choice(tipoTransaccion),
        'Cuenta origen' : random.choice(cuentas),
        'Cuenta destino' : random.choice(cuentas),
        'Fecha' : random.choice(fechas),
        'Hora' : random.choice(horas),
        'Monto' : random.uniform(0.0, 1000.0)
    }, ignore_index = True)
datos


,Num. de transaccion,Tipo de transaccion,Cuenta origen,Cuenta destino,Fecha,Hora,Monto
0,15,NaN,Cuenta 5,Cuenta 3,7/6/2019,15:45:00,394.391659
1,16,Transferencia,Cuenta 1,Cuenta 5,7/6/2019,15:45:00,899.873770
2,17,Transferencia,Cuenta 5,Cuenta 1,23/9/2018,15:45:00,698.130618
3,18,NaN,Cuenta 2,Cuenta 2,7/6/2019,13:05:00,78.224078
4,19,NaN,Cuenta 1,Cuenta 3,20/6/2019,13:05:00,367.751510
5,20,Extraccion,Cuenta 5,NaN,20/6/2019,13:05:00,17.780608
6,21,NaN,Cuenta 4,Cuenta 1,2/10/2019,10:00:00,951.135311
7,22,Transferencia,NaN,Cuenta 2,7/6/2019,13:05:00,991.293581
8,23,NaN,Cuenta 5,Cuenta 4,20/6/2019,16:23:05,694.907539
9,24,NaN,Cuenta 4,Cuenta 2,5/1/2020,16:23:05,36.777931


In [2]:
#A) Validar que todas las transacciones cuenten con un tipo de transaccion

datos['Tipo de transaccion'].isnull()

0     True
1    False
2    False
3     True
4     True
5    False
6     True
7    False
8     True
9     True
Name: Tipo de transaccion, dtype: bool

In [3]:
#B) validar que para las transacciones del tipo transferencia, exista siempre tanto cuenta origen como cuenta 
#destino

#me quedo con las transferencias
transferencias = datos[datos['Tipo de transaccion'] == 'Transferencia']

#chequeo si las cuentas origen y destino son distintas a NaN
(transferencias['Cuenta origen'].isnull() | transferencias['Cuenta destino'].isnull())


1    False
2    False
7     True
dtype: bool

In [4]:
#C) Verificar que odas las transacciones del tipo transefrencia, deposito y extraccion cuenten con montos
#distintos de cero

nuevoDf = datos.copy()
#Agrego la columna "Montos != 0" con los montos del viejo df
nuevoDf['Montos != 0'] = nuevoDf['Monto'] != 0

#agrupo por tipo de transaccion, agregando como monto != 0 un booleano
nuevoDf.groupby('Tipo de transaccion').agg({'Montos != 0': 'any'})

,Montos != 0
Tipo de transaccion,
Extraccion,True
Transferencia,True


In [5]:
#D) Indicar cuales fueron las 10 transacciones de mayor monto

#agrupo por tipo de transaccion, verificando que no sea NaN y busco los 10 montos mayores
datos[~datos['Tipo de transaccion'].isnull()].nlargest(10,'Monto')

,Num. de transaccion,Tipo de transaccion,Cuenta origen,Cuenta destino,Fecha,Hora,Monto
7,22,Transferencia,NaN,Cuenta 2,7/6/2019,13:05:00,991.293581
1,16,Transferencia,Cuenta 1,Cuenta 5,7/6/2019,15:45:00,899.873770
2,17,Transferencia,Cuenta 5,Cuenta 1,23/9/2018,15:45:00,698.130618
5,20,Extraccion,Cuenta 5,NaN,20/6/2019,13:05:00,17.780608


In [6]:
#E) Indicar cual es el tipo de transaccion con mayor monto promedio

#agrupo por tipo de transaccion y agrego el promedio como monto
datos.groupby('Tipo de transaccion').agg({'Monto' : 'mean'}).nlargest(1, 'Monto')

,Monto
Tipo de transaccion,
Transferencia,863.099323


In [7]:
#F) Indicar cuales son las 5 cuentas con mayor cantidad de transacciones

#agrupo por cuentas de origen y como numero de transaccion, agrego la suma total de las transacciones
#de esa cuenta
cantidadTransaccionesPorCuenta = datos[~datos['Cuenta origen'].isnull()].groupby('Cuenta origen').agg({'Num. de transaccion' : 'count'})

#Cambio el nombre de la columna a Cantidad de transacciones
cantidadTransaccionesPorCuenta.columns = ['Cantidad de transacciones']

#Me quedo con las primeras 5
cantidadTransaccionesPorCuenta.nlargest(5, 'Cantidad de transacciones')

,Cantidad de transacciones
Cuenta origen,
Cuenta 5,4
Cuenta 1,2
Cuenta 4,2
Cuenta 2,1


In [8]:
#G) Indicar cuales son las 5 cuentas con mayor monto involucrado

montoMayorPorCuenta = datos[~datos['Cuenta origen'].isnull()].groupby('Cuenta origen').agg({'Monto' : 'sum'})
montoMayorPorCuenta.nlargest(5, 'Monto')


,Monto
Cuenta origen,
Cuenta 5,1805.210424
Cuenta 1,1267.625280
Cuenta 4,987.913242
Cuenta 2,78.224078


In [9]:
#H) Para el tipo de transaccion con mayor cantidad de monto promedio, indicar cuales son las
#5 cuentas con mas transaacciones

#agrupo por tipo de transaccion y agrego el promedio como monto

tipoDeTransaccionMayorMonto = datos.groupby('Tipo de transaccion').agg({'Monto' : 'mean'}).nlargest(1, 'Monto')
tipoDeTransaccionMayorMonto.columns = ['Monto promedio'] 
tipoDeTransaccionMayorMonto = tipoDeTransaccion.reset_index()
tipoDeTransaccionMayorMonto = tipoDeTransaccionMayorMonto.loc[0, 'Tipo de transaccion']

#busco cuentas con mas transacciones de mayor monto
cuentasConMasTransacciones = datos[(datos['Tipo de transaccion'] == tipoDeTransaccionMayorMonto) & (~datos['Cuenta origen'].isnull())]
cuentasConMasTransacciones = cuentasConMasTransacciones.groupby('Cuenta origen').agg({'Num. de transaccion' : 'count'}).nlargest(5, 'Num. de transaccion')
cuentasConMasTransacciones.columns = ['Cantidad de transacciones']
cuentasConMasTransacciones

NameError: name 'tipoDeTransaccion' is not defined